In [ ]:
import sys

# Add Robyn to path
sys.path.append("/Users/yijuilee/robynpy_release_reviews/Robyn/python/src")

In [ ]:
# Test Pareto Optimizer

from robyn.modeling.pareto.pareto_optimizer import ParetoOptimizer
from robyn.tutorials.utils.data_mapper import (
    import_output_models,
    import_input_collect,
    load_data_from_json,
)

In [ ]:
# Load data from JSON
inputCollect = load_data_from_json(
    "/Users/yijuilee/project_robyn/original/Robyn_original_2/Robyn/robyn_api/data/Pareto_InputCollect.json"
)
outputModel = load_data_from_json(
    "/Users/yijuilee/project_robyn/original/Robyn_original_2/Robyn/robyn_api/data/Pareto_OutputModels.json"
)
input_collect = import_input_collect(inputCollect)
output_models = import_output_models(outputModel)

In [ ]:
mmm_data = input_collect["mmm_data"]
# display(mmm_data.data.head())
# Display Model Outputs

output_models = output_models
# display((model_outputs.trials[0].result_hyp_param))

hyperparameters = input_collect["hyperparameters"]
# display(hyperparameters)

featurized_mmm_data = input_collect["featurized_mmm_data"]

holidays_data = input_collect["holidays_data"]

In [ ]:
# Add after model training
print("Model Output Summary:")
print(f"Number of trials: {len(output_models.trials)}")
print(
    f"Average models per trial: {len(output_models.all_result_hyp_param) / len(output_models.trials)}"
)
print(
    f"Total unique models: {len(output_models.all_result_hyp_param['sol_id'].unique())}"
)

print("\nMetrics Distribution:")
metrics_df = output_models.all_result_hyp_param[["nrmse", "decomp.rssd", "mape"]]
print(metrics_df.describe())

# Additional validation to debug model output
print("\nColumns in result_hyp_param:")
print(output_models.all_result_hyp_param.columns.tolist())

print("\nSample rows of metrics:")
print(
    output_models.all_result_hyp_param[
        ["sol_id", "nrmse", "decomp.rssd", "mape"]
    ].head()
)

# Show shape of result dataframes
print("\nDataFrame Shapes:")
print(f"result_hyp_param: {output_models.all_result_hyp_param.shape}")
print(f"x_decomp_agg: {output_models.all_x_decomp_agg.shape}")
print(f"decomp_spend_dist: {output_models.all_decomp_spend_dist.shape}")

In [ ]:
# 3. Create ParetoOptimizer instance
pareto_optimizer = ParetoOptimizer(
    mmm_data, output_models, hyperparameters, featurized_mmm_data, holidays_data
)

In [ ]:
# 4. Run optimize function
pareto_result = pareto_optimizer.optimize(pareto_fronts="auto", min_candidates=100)

In [ ]:
# 5. Check results
print("Pareto Optimization Results:")
print(
    f"Number of Pareto fronts: {len(pareto_result.pareto_solutions)} - {pareto_result.pareto_solutions}"
)
print(
    f"MediaVecCollect: {pareto_result.media_vec_collect.shape, pareto_result.media_vec_collect}"
)
print("\Hyper parameter solutions:")
print(pareto_result.result_hyp_param)

print("\nAggregated decomposition results:")
print(pareto_result.x_decomp_agg)
print("\result Calibration:")
print(pareto_result.result_calibration)
print("\nx Decomp Vec Collect:")
print(pareto_result.x_decomp_vec_collect.shape, pareto_result.x_decomp_vec_collect)
print("\nCarryover percentage all:")
print(pareto_result.df_caov_pct_all.shape, pareto_result.df_caov_pct_all)
print("\Plot Data Collected")
# print("NUMBER OF PLOTS Data collected for:", len(pareto_result.plot_data_collect["2_4_8"]))
# print("Plot data for solid 3_206_6", pareto_result.plot_data_collect["2_4_8"])

# 6. Validate logic
assert pareto_result.pareto_fronts == "auto" or isinstance(
    pareto_result.pareto_fronts, int
), "Invalid pareto_fronts value"
assert not pareto_result.result_hyp_param.empty, "Empty result_hyp_param DataFrame"
assert not pareto_result.x_decomp_agg.empty, "Empty x_decomp_agg DataFrame"

print("\nAll assertions passed. The optimize function is working as expected.")

In [ ]:
print(pareto_result.x_decomp_agg[pareto_result.x_decomp_agg["sol_id"] == "5_221_9"])

# Clustering

In [ ]:
from robyn.modeling.clustering.clustering_config import ClusteringConfig, ClusterBy
from robyn.modeling.clustering.cluster_builder import ClusterBuilder
from robyn.data.entities.enums import DependentVarType
import plotly.io as pio

pio.renderers.default = "iframe"

cluster_configs = ClusteringConfig(
    dep_var_type=DependentVarType(mmm_data.mmmdata_spec.dep_var_type),
    cluster_by=ClusterBy.HYPERPARAMETERS,
    max_clusters=10,
    min_clusters=3,
    weights=[1.0, 1.0, 1.0],
)

cluster_builder = ClusterBuilder(pareto_result=pareto_result)

In [ ]:
cluster_results = cluster_builder.cluster_models(cluster_configs)

In [ ]:
# Cluster result Validations (graphs)
print(cluster_results.top_solutions["sol_id"])
cluster_results.wss

In [ ]:
cluster_results.plots.top_solutions_errors_plot

In [ ]:
cluster_results.plots.top_solutions_rois_plot

In [ ]:
cluster_results.cluster_ci.clusters_confidence_interval_plot

### Process Pareto Clustered Results for Allocator

In [ ]:
from robyn.modeling.pareto.pareto_utils import ParetoUtils

utils = ParetoUtils()
filtered_pareto_results = utils.process_pareto_clustered_results(
    pareto_result,
    clustered_result=cluster_results,
    ran_cluster=True,
    ran_calibration=False,
)

# One Pager Results

In [ ]:
from robyn.data.entities.enums import AdstockType
from robyn.reporting.onepager_reporting import OnePager
from robyn.visualization.pareto_visualizer import ParetoVisualizer


visualizer = OnePager(
    pareto_result=filtered_pareto_results,
    clustered_result=cluster_results,
    hyperparameter=hyperparameters,
    mmm_data=mmm_data,
    holidays_data=holidays_data,
)
visualizer.generate_one_pager(top_pareto=True)

In [ ]:
visualizer = ParetoVisualizer(
    pareto_result=filtered_pareto_results,
    hyperparameter=hyperparameters,
    mmm_data=mmm_data,
    holiday_data=holidays_data,
    featurized_mmm_data=featurized_mmm_data,
    unfiltered_pareto_result=pareto_result,
    model_outputs=output_models,
)

visualizer.plot_all(True)

# Allocator